# 📚 EPUB Compressor для Kindle

Сжимает EPUB файлы с сохранением качества ~90% для передачи на Kindle

**Просто загрузите ваш EPUB файл и получите сжатый результат!**


In [ ]:
# Установка зависимостей
!pip install Pillow -q
print("✅ Зависимости установлены!")


In [ ]:
import zipfile
import os
import io
from PIL import Image
from pathlib import Path
from google.colab import files
import shutil

# Поддерживаемые форматы изображений
IMAGE_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp'}
MAX_IMAGE_WIDTH = 1920  # Максимальная ширина для Kindle
MAX_IMAGE_HEIGHT = 2560  # Максимальная высота для Kindle
JPEG_QUALITY = 85  # Качество JPEG (85-90 для сохранения ~90% качества)

print("✅ Библиотеки загружены!")


In [ ]:
def get_file_size_mb(filepath):
    """Получить размер файла в МБ"""
    return os.path.getsize(filepath) / (1024 * 1024)


def optimize_image(image_data, filename):
    """
    Оптимизирует изображение
    Возвращает оптимизированные данные изображения
    """
    try:
        # Открываем изображение из байтов
        img = Image.open(io.BytesIO(image_data))
        original_format = img.format
        original_size = img.size
        
        # Конвертируем RGBA в RGB для JPEG
        if img.mode in ('RGBA', 'LA', 'P'):
            # Создаем белый фон для прозрачных изображений
            background = Image.new('RGB', img.size, (255, 255, 255))
            if img.mode == 'P':
                img = img.convert('RGBA')
            background.paste(img, mask=img.split()[-1] if img.mode in ('RGBA', 'LA') else None)
            img = background
        elif img.mode != 'RGB':
            img = img.convert('RGB')
        
        # Уменьшаем размер если слишком большой
        width, height = img.size
        if width > MAX_IMAGE_WIDTH or height > MAX_IMAGE_HEIGHT:
            img.thumbnail((MAX_IMAGE_WIDTH, MAX_IMAGE_HEIGHT), Image.Resampling.LANCZOS)
        
        # Сохраняем в JPEG для лучшего сжатия
        output = io.BytesIO()
        img.save(output, format='JPEG', quality=JPEG_QUALITY, optimize=True)
        output.seek(0)
        
        new_size = len(output.getvalue())
        original_size_bytes = len(image_data)
        compression_ratio = (1 - new_size / original_size_bytes) * 100
        
        print(f"  ✓ {filename} ({original_size[0]}x{original_size[1]}) "
              f"{original_size_bytes/1024:.1f}KB → {new_size/1024:.1f}KB "
              f"({compression_ratio:.1f}% сжатие)")
        
        return output.getvalue()
    
    except Exception as e:
        print(f"  ⚠ Ошибка при оптимизации {filename}: {e}")
        return image_data  # Возвращаем оригинал при ошибке


def compress_epub(input_file, output_file, target_size_mb=49):
    """
    Сжимает EPUB файл
    """
    if not os.path.exists(input_file):
        print(f"❌ Ошибка: файл {input_file} не найден")
        return False
    
    original_size = get_file_size_mb(input_file)
    print(f"📊 Исходный размер: {original_size:.2f} МБ")
    print(f"🎯 Целевой размер: {target_size_mb} МБ")
    print(f"📉 Необходимое сжатие: {(1 - target_size_mb/original_size)*100:.1f}%")
    print("\n⏳ Начинаю обработку...\n")
    
    try:
        # Открываем исходный EPUB
        with zipfile.ZipFile(input_file, 'r') as input_zip:
            # Создаем новый EPUB
            with zipfile.ZipFile(output_file, 'w', zipfile.ZIP_DEFLATED, compresslevel=9) as output_zip:
                total_files = len(input_zip.namelist())
                processed_files = 0
                images_optimized = 0
                total_original_size = 0
                total_compressed_size = 0
                
                for file_info in input_zip.infolist():
                    filename = file_info.filename
                    processed_files += 1
                    
                    # Пропускаем служебные файлы macOS
                    if filename.startswith('__MACOSX/') or filename.endswith('.DS_Store'):
                        continue
                    
                    file_data = input_zip.read(filename)
                    original_file_size = len(file_data)
                    total_original_size += original_file_size
                    
                    # Проверяем, является ли файл изображением
                    file_ext = Path(filename).suffix.lower()
                    if file_ext in IMAGE_EXTENSIONS:
                        if processed_files % 10 == 0 or processed_files == 1:
                            print(f"[{processed_files}/{total_files}] Обработка изображений...")
                        optimized_data = optimize_image(file_data, filename)
                        total_compressed_size += len(optimized_data)
                        images_optimized += 1
                        output_zip.writestr(file_info, optimized_data)
                    else:
                        # Для не-изображений просто копируем
                        total_compressed_size += original_file_size
                        output_zip.writestr(file_info, file_data)
                
                print(f"\n{'='*60}")
                print(f"✅ Обработано файлов: {processed_files}")
                print(f"🖼️  Оптимизировано изображений: {images_optimized}")
                print(f"📦 Исходный размер: {total_original_size / (1024*1024):.2f} МБ")
                print(f"📦 Размер после сжатия: {total_compressed_size / (1024*1024):.2f} МБ")
        
        # Проверяем финальный размер
        final_size = get_file_size_mb(output_file)
        compression_achieved = (1 - final_size/original_size) * 100
        
        print(f"\n{'='*60}")
        print(f"📊 РЕЗУЛЬТАТ:")
        print(f"📥 Исходный размер: {original_size:.2f} МБ")
        print(f"📤 Финальный размер: {final_size:.2f} МБ")
        print(f"📉 Достигнутое сжатие: {compression_achieved:.1f}%")
        
        if final_size <= target_size_mb:
            print(f"\n✅ Цель достигнута! Файл меньше {target_size_mb} МБ")
        else:
            print(f"\n⚠️  Файл все еще больше {target_size_mb} МБ")
            print(f"💡 Можно попробовать уменьшить параметры сжатия")
        
        return True
    
    except Exception as e:
        print(f"❌ Ошибка при обработке: {e}")
        import traceback
        traceback.print_exc()
        return False

print("✅ Функции загружены!")


## 📤 Шаг 1: Загрузите ваш EPUB файл

Нажмите кнопку ниже и выберите ваш EPUB файл


In [ ]:
# Загрузка файла
uploaded = files.upload()

if not uploaded:
    print("❌ Файл не загружен. Пожалуйста, попробуйте снова.")
else:
    # Получаем имя загруженного файла
    input_filename = list(uploaded.keys())[0]
    print(f"✅ Файл загружен: {input_filename}")
    print(f"📊 Размер: {get_file_size_mb(input_filename):.2f} МБ")


## ⚙️ Шаг 2: Настройте параметры сжатия (опционально)

Вы можете изменить параметры ниже, если нужно более агрессивное сжатие:


In [ ]:
# Настройки сжатия (можно изменить)
TARGET_SIZE_MB = 49  # Целевой размер в МБ

# Для более агрессивного сжатия раскомментируйте:
# MAX_IMAGE_WIDTH = 1600
# MAX_IMAGE_HEIGHT = 2000
# JPEG_QUALITY = 80

print(f"🎯 Целевой размер: {TARGET_SIZE_MB} МБ")


## 🔄 Шаг 3: Запустите сжатие

Нажмите кнопку запуска ниже для начала обработки


In [ ]:
# Обработка файла
if 'input_filename' in locals():
    # Создаем имя выходного файла
    path = Path(input_filename)
    output_filename = f"{path.stem}_compressed{path.suffix}"
    
    print("="*60)
    print("🚀 EPUB Compressor для Kindle")
    print("="*60)
    print()
    
    success = compress_epub(input_filename, output_filename, TARGET_SIZE_MB)
    
    if success:
        print(f"\n✅ Готово! Файл сохранен как: {output_filename}")
    else:
        print("\n❌ Произошла ошибка при обработке")
else:
    print("❌ Сначала загрузите файл в ячейке выше!")


## 📥 Шаг 4: Скачайте сжатый файл

Нажмите кнопку ниже для скачивания результата


In [ ]:
# Скачивание результата
if 'output_filename' in locals() and os.path.exists(output_filename):
    files.download(output_filename)
    print(f"✅ Файл {output_filename} готов к скачиванию!")
else:
    print("❌ Файл не найден. Убедитесь, что вы выполнили шаг 3.")
